In [1]:
import jupyter_black

jupyter_black.load()

%load_ext autoreload
%autoreload 2

In [2]:
import time

import gymnasium as gym
import torch
import torch.nn as nn
import numpy as np
from torch.distributions import Categorical

from ppo import PPO

# Initialise the environment
env = gym.make("CartPole-v1")

In [3]:
input_dim = env.observation_space.shape[0]
out_dim = env.action_space.n
hidden_dim = 16

In [4]:
actor = nn.Sequential(
    nn.Linear(in_features=input_dim, out_features=hidden_dim),
    nn.Linear(in_features=hidden_dim, out_features=out_dim),
    nn.Softmax(dim=1),
)
critic = nn.Sequential(
    nn.Linear(in_features=input_dim, out_features=hidden_dim),
    nn.Linear(in_features=hidden_dim, out_features=1),
)

In [5]:
ppo = PPO(
    env=env,
    actor=actor,
    critic=critic,
    max_training_samples=2048,
    gamma=0.99,
    epsilon=0.2,
    batch_size=64,
    num_epochs=10,
    num_training_cycles=30,
)

In [6]:
ppo.train()

  0%|          | 0/30 [00:00<?, ?it/s]

Average episode length: 22.051546391752577


  3%|▎         | 1/30 [00:01<00:52,  1.80s/it]

Average episode length: 17.296


  7%|▋         | 2/30 [00:03<00:49,  1.75s/it]

...

 93%|█████████▎| 28/30 [00:46<00:03,  1.67s/it]

Average episode length: 31.029411764705884


 97%|█████████▋| 29/30 [00:48<00:01,  1.67s/it]

Average episode length: 18.612068965517242


100%|██████████| 30/30 [00:50<00:00,  1.68s/it]


In [11]:
env = gym.make("CartPole-v1", render_mode="human")

In [12]:
# Set model to eval mode
ppo.critic.eval()

# Reset environment
obs, _ = env.reset()
done = False

while not done:
    # Convert observation to tensor
    state = torch.from_numpy(obs).float().unsqueeze(0)  # shape: (1, 4)

    # Get value estimate from critic
    with torch.no_grad():
        value = ppo.critic(state).item()

    # Pick action randomly or use a policy (if you have ppo.actor)
    action = env.action_space.sample()

    # Step in environment
    obs, reward, terminated, truncated, _ = env.step(action)
    done = terminated or truncated

    time.sleep(0.05)  # Slow down for visualization

env.close()